In [35]:
import os
import json
import openai
import requests
from pptx.util import Inches
from pptx import Presentation
from concurrent.futures import ThreadPoolExecutor, as_completed

client = openai.OpenAI()

In [ ]:
# Generate Content for Presentation and structure it in json format
# Concurrently Generate Image Descriptions to be used to generate images 

system_prompt = """ You excel in crafting detailed outlines for presentations, demonstrating expertise in structuring content effectively.
"""

user_prompt = r""" Compose high-level outlines to guide PowerPoint creation.

The Topic on which you have to create outline is: **Funding Pitch Deck For Commercial Real Estate**

Below are the Details Reagarding the Same:
```
Company Name: OutSite Estates
Company Specility: Devloping Luxury Hotels and Places on popular Tourist Destinations - Especially Natural toursit destinations
Founded: 2020
Revenue Per Year: 1.5M USD
Profits: 100K USD
```

```json
{
    "slides": [
        {
            "header": "Head of the slide",
            "content": "Brief Description in the Slide",
            "image": "Description of appropriate image to be put in slide"
        },
        ...
    ]
}
```
"""

completion = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    temperature=0.1,
    response_format={ "type": "json_object" },
)

slides_data = json.loads(completion.choices[0].message.content)
image_prompts = [slide['image'] for slide in slides_data['slides']]

In [ ]:
# Concurrently generate all the images for the slides with Dall-e
# And save them in generated images directory

def generate_image(index, image_prompt, save_directory):
    # Generate the image using the DALL-E API
    response = client.images.generate(
        model="dall-e-3",
        prompt=image_prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )

    # Extract the URL of the generated image
    image_url = response.data[0].url

    # The filename for the image, using the index
    image_filename = f"{index}.png"
    save_path = os.path.join(save_directory, image_filename)

    # Make a GET request to the image URL
    response = requests.get(image_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Write the image content to a file in the specified directory
        with open(save_path, 'wb') as image_file:
            image_file.write(response.content)
        print(f"Image successfully downloaded to {save_path}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")

    return save_path


# The directory where the images will be saved
save_directory = "generated_images"
os.makedirs(save_directory, exist_ok=True)  

generated_images = []
max_workers = 20  # Adjust the number of workers as needed

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = []

    for index, image_prompt in enumerate(image_prompts):
        future = executor.submit(generate_image, index, image_prompt, save_directory)
        futures.append(future)

    for future in as_completed(futures):
        try:
            result = future.result()
            generated_images.append(result)
        except Exception as exc:
            raise ValueError("Error")

In [ ]:
# Combine the paths of generated images with the previously generated JSON
assert len(slides_data['slides']) == len(generated_images), "The number of slides and image paths must match."


for slide, img_path in zip(slides_data['slides'], sorted(generated_images, key=lambda x: int(x.split('/')[-1].split('.')[0]))):
    slide['image_path'] = img_path

# optionally save the json for later use
with open("slides_data.json", 'w') as file:
    json.dump(slides_data, file, indent=4)

In [43]:
# Create Presentation with the generated JSON and images

prs = Presentation()

# Loop through each slide in the JSON
for slide_info in slides_data['slides']:
    # Add a slide with a title and content layout
    slide_layout = prs.slide_layouts[5]  # Using a blank slide layout for custom placement
    slide = prs.slides.add_slide(slide_layout)
    
    # Set the title for the slide
    title_shape = slide.shapes.title
    title_shape.text = slide_info['header']
    
    # Calculate the position and size for the text box and image
    left = Inches(0.5)  # Left margin
    top = Inches(2)     # Top margin for the text box
    width = Inches(4.5) # Width of the text box
    height = Inches(3)  # Height of the text box
    
    # Add the text box
    textbox = slide.shapes.add_textbox(left, top, width, height)
    text_frame = textbox.text_frame
    p = text_frame.add_paragraph()
    p.text = slide_info['content']
    
    # Calculate the position for the image
    image_left = Inches(3) 
    image_top = Inches(3)
    image_width = Inches(4)   
    image_height = Inches(4)  
    
    # Add the image
    img_path = slide_info['image_path']
    slide.shapes.add_picture(img_path, image_left, image_top, image_width, image_height)

# Save the presentation
prs.save('generated_presentation_with_images.pptx')